### Cross Encoder를 알아야 하는 이유

- 문제해결 관점에서 비슷한 분야의 문서를 분류하거나, 궁금한 내용을 찾아야 하는 경우, 욕설 문장을 비교하거나 등등.. 필요한 경우가 있음.

- 문장 간 관계를 1:1 비교하는 방법임.

- Q&A, STS, NLI Task 모두 Cross Encoder기반으로 문제 해결가능

- 문장과 문장을 1대1로 비교해 연관성을 측정하는 방법임

- STS의 경우 문장 1:1로 비교해 유사도를 0~5 범위로 output 반환

- NLI의 경우도 문장을 1:1로 비교해 Entailment, Neutral, Contradiction으로 Output 반환

- Q&A의 경우 Question과 DataBase 내 자료를 1:1로 비교 Question 1개, Database 내 10개의 문장이 있다고 가정할 때, (Question, corpus1), (Question, corpus2) ... (Question, corpus10)과 같이 10번을 연산 한 결과를 비교해 가장 높은 값을 채택

- 문장의 유사도는 번역, 요약, 문장 생성, QA, 대화 모델링 등등 다양한 NLP 분야에서 중요하게 다뤄진다

<!-- * Cross Encoder를 코드로 구현하며 코드 내부 데이터 흐름에 대한 설명을 이어나가겠음. -->

> 이번 글에서는 Cross Encoder 구조 이해를 위해 Base Model 위에 Cross-encoder layer를 쌓고 이를 학습하는 방법을 설명함.

### Bi encoder와 Cross encoder 비교

- Bi Decoder는 Pooling을 통해 여러 개 토큰으로 구성된 문장을 하나의 토큰으로 압축. 이러한 방법으로 DB내 모든 문장을 백터화하여 저장 해놓으면 cosine similiarity를 활용해 다양한 Task를 수행할 수 있음. 수십개의 토큰을 하나의 토큰으로 바꾸는 방법이므로 정확도가 낮아지는 단점이 있지만 모든 문장을 하나의 Vector Space 배치하므로 연산속도면에서 장점이 있음

- Cross Encoder는 문장을 1:1로 비교해야하는 단점이 있지만 문장 내 모든 토큰을 활용해 연관성을 파악할 수 있으므로 정확도 면에서 장점이 있음.

- 하나의 Encoder만 사용하는 경우는 거의 없고 주로 Bi encoder로 우선순위가 높은 문장 50~100개를 추출한 뒤 Cross Encoder를 사용해 상세 순위를 비교하는 방법으로 사용함.

<img src='img/Bi_vs_Cross-Encoder.png' alt='comparsion'>


In [ ]:
import torch
from transformers import ElectraModel, ElectraTokenizerFast

model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")
tokenizer = ElectraTokenizerFast.from_pretrained(
    "monologg/koelectra-base-v3-discriminator"
)


### 데이터 불러오기


### Cross encoder 구조 살펴보기

- Cross encoder는 Model의 Last-hidden-state에 Label 개수에 맞는 output을 반환하는 classifier를 얹은 구조임.

- Huggingface의 Sequenceclassification model을 불러오면 쉽게 사용이 가능함.

- Sbert에서 제공하는 Cross Encoder도 SequenceClassification 구조를 사용하고 있음.

- 따라서 해당 모델의 구조를 살펴봄으로서 Cross Encoder의 구조와 코드 구현 방법에 대해 설명하겠음


#### ClassificationHead

- Sequence Classification 모델은 Classification Class와 Electra Model Class를 하나로 합친 모델이다.

- Classification 구조를 보면 dense_layer => gelu => output_pojection_layer 로 되어있음.

- Model의 last hidden layer output 중 [CLS] 토큰만을 활용함.

- Model Output은 [batch_size, src_token, embed_size]이 되고 그중 [CLS] 토큰만 활용하므로 [batch_size embed_size]으로 차원이 감소함. 이를 Pooling 한다고 하며 Denselayer와 gelu(Electra에선 gelu, Bert에선 tanh)를 거침 [Why it called pooler? 참고](https://github.com/google-research/bert/issues/1102)

- 이후 output_prj_layer를 통해 Label 맞는 차원으로 감소시킴. Regression 모델인 경우 Label size를 1로, Classification 모델인 경우 분류에 필요한 Label 개수에 맞게 설정해야함.


In [ ]:
from torch import Tensor, nn
from torch.nn import BCEWithLogitsLoss
from transformers import ElectraPreTrainedModel, ElectraForSequenceClassification
from typing import Optional


class classificationHead(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        classifier_dropout = (
            config.classifier_dropout
            if config.classifier_dropout is not None
            else config.hidden_dropout_prob
        )
        self.gelu = nn.functional.gelu

        self.dropout = nn.Dropout(classifier_dropout)

        # [batch, embed_size] => [batch, num_labels]
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        # [CLS] 토큰 추출 [batch, src_token, embed_size] => [batch, embed_size]
        x = features[:, 0, :]
        x = self.dropout(x)
        x = self.dense(x)
        x = self.gelu(x)
        x = self.dropout(x)

        # label 개수만큼 차원 축소 [batch, embed_size] => [batch, num_labels]
        x = self.out_proj(x)

        return x


### sequenceClassification

- ElectraWithClassification은 Electra Model의 Output을 위에서 정의한 Classifier로 연결한 모델임.

- 설명을 돕기 위해 ElectraWithClassification을 임의로 만들었으며 ElectraForSequenceClassification 내부 코드를 이해하기 쉽게 일부 변형하였음.

- Output은 Label이 모델 내 제공되는 경우(=모델 학습 시) Loss와 Logits으로 반환하고, Label이 제공되지 않는 경우(=평가 시) Logits만 반환함.

- Loss function은 학습 유형이 Regression일 때 MSE, Single-Classfication일 때 Cross-Enctropy, Multi-Classification일 때 bcewithlogitsloss를 활용함.
  - 학습 유형에 따라 Loss Function이 달라지는 이유에 대해선 [In which cases is the cross-entropy preferred over the mean squared error?](https://stackoverflow.com/questions/36515202/in-which-cases-is-the-cross-entropy-preferred-over-the-mean-squared-error)와 [What is the different between MSE error and Cross-entropy error in NN](https://susanqq.github.io/tmp_post/2017-09-05-crossentropyvsmes/)를 참고


In [ ]:
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss


class ElectraWithClassification(nn.Module):
    def __init__(self, model, num_labels) -> None:
        super().__init__()
        self.model = model
        self.model.config.num_labels = num_labels
        self.classifier = classificationHead(self.model.config)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):

        # Last-hidden-states 연산
        discriminator_hidden_states = self.model(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = discriminator_hidden_states[0]

        # classificationHead에 Last-hidden-state 대입
        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None:
            # Multi-classification 인 경우
            # loss_fct = BCEWithLogitsLoss()

            # # Regression 인 경우
            # loss_fct = MSELoss()
            # Single-classfication 인 경우
            loss_fct = CrossEntropyLoss()

            # _, logits = torch.max(logits, dim=1)
            # print(logits.float())
            # print(labels.float())
            loss = loss_fct(logits.view(-1, 3), labels.view(-1))
            return {"loss": loss, "logit": logits}
        else:
            return {"logit": logits}


### 모델 학습


### KorNLI를 활용해 모델 학습 시키겠음


### Training dataset


In [ ]:
import pandas as pd

with open("data/KorNLI/snli_1.0_train.ko.tsv") as f:
    v = f.readlines()

## from list to dataframe
lst = [i.rstrip("\n").split("\t") for i in v]

data = pd.DataFrame(lst[1:], columns=lst[:1])
data.columns = ["sen1", "sen2", "gold_label"]
data.head(3)


### Gold_label Encoding


In [ ]:
label2int = {"contradiction": 0, "entailment": 1, "neutral": 2}

data["gold_label"] = data["gold_label"].replace(label2int).values

data.head(3)


In [ ]:
from datasets import Dataset

train_data_set = Dataset.from_pandas(data)

train_data_set[0]


### Eval dataset


In [ ]:
with open("data/KorNLI/xnli.dev.ko.tsv") as f:
    v = f.readlines()

## from list to dataframe
lst = [i.rstrip("\n").split("\t") for i in v]

data = pd.DataFrame(lst[1:], columns=lst[:1])
data.columns = ["sen1", "sen2", "gold_label"]
data.head(3)


In [ ]:
label2int = {"contradiction": 0, "entailment": 1, "neutral": 2}

data["gold_label"] = data["gold_label"].replace(label2int).values

data.head(3)


In [ ]:
from datasets import Dataset

eval_data_set = Dataset.from_pandas(data)

eval_data_set[0]


In [ ]:
def smart_batching_collate(batch):
    text_lst1 = []
    text_lst2 = []
    labels = []

    for example in batch:
        for k, v in example.items():
            if k == "sen1":
                text_lst1.append(v)
            if k == "sen2":
                text_lst2.append(v)
            if k == "gold_label":
                labels.append(int(v))

    token = tokenizer(
        text_lst1,
        text_lst2,
        return_tensors="pt",
        truncation=True,
        padding=True,
    )

    return dict(**token, labels=torch.LongTensor(labels))


### Trainer


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_trainer",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_steps=10,
    eval_steps=10,
    num_train_epochs=2,
    remove_unused_columns=False,
    evaluation_strategy="steps",
    save_steps=2000,
)

# from transformers import ElectraForSequenceClassification

# cross_encoder = ElectraForSequenceClassification.from_pretrained('model/disc_book_final',num_labels=3)
# or
cross_encoder = ElectraWithClassification(model=model, num_labels=3)

trainer = Trainer(
    model=cross_encoder,
    train_dataset=train_data_set,
    eval_dataset=eval_data_set,
    args=training_args,
    data_collator=smart_batching_collate,
)

trainer.train()

# trainer.evaluate()
